# Streams and Roofs

In this week's assignment we are going to make some roofline diagrams for some $n$-body problems.

This week's assignment is meant to be run on a node with a Tesla P100 GPU.

A reminder: when you are running a job to complete this week's assignment, you should make sure that you have requested exclusive access to a node, and that you have requested access to all CPU cores of this node.

**Due: Thursday, September 12, before class**

Let's load in our class module:

In [1]:
!module use $CSE6230_DIR/modulefiles

In [2]:
!module load cse6230

In [3]:
!module list

Currently Loaded Modulefiles:
  1) curl/7.42.1
  2) git/2.13.4
  3) python/3.6
  4) /nv/coc-ice/tisaac3/opt/pace-ice/modulefiles/jupyter/1.0
  5) intel/16.0
  6) cuda/8.0.44
  7) /nv/coc-ice/tisaac3/opt/pace-ice/modulefiles/hpctoolkit/2018.18
  8) impi/5.1.1.109
  9) cse6230/core(default)


And verify that we're running where we expect to run:

In [4]:
!nvidia-smi

Wed Sep 11 22:04:56 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.30                 Driver Version: 390.30                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  On   | 00000000:81:00.0 Off |                    0 |
| N/A   27C    P0    25W / 250W |      0MiB / 16280MiB |      0%   E. Process |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Great!

Now, about the $n$-body simulations we're going to run: a classical $n$-body simulation has each body, or *particle*, interacting with each other, for $n(n+1)/2$ total interactions.  That hardly matches up to the streaming kernels we've been talking about!  So we're going to simplify a bit.

We are going to simulate $n$ infinitesimal particles circling around an infinitely massive sun at the origin.  In this system, the sun is unmoved, and the particles are not affected by each other.

We're going to normalize our coefficients and say that each particle is an ordinary differential equation with *six* components: three of position $X=(x, y, z)$ and three of velocity $U=(u, v, w)$.  The position, is changed by the velocity, of course, but the velocity changes under acceleration that depends on position:

$$\begin{aligned} \dot{X} &= V \\ \dot{V} &= - \frac{X}{|X|^3}.\end{aligned}$$

To discretize this differential equation, we are going to use a time stepping method called the Verlet leap-frog method, which is good for calculating long simulations of stable orbits.  Given a time step length `dt`, our pseudocode for one time step for one particle looks like the following:

1. `X += 0.5 * dt * V`
2. `R2 = X . X` (dot product)
3. `R = sqrt (R2)`
4. `IR3 = 1. / (R2 * R)`
5. `V -= X * dt * IR3`
6. `X += 0.5 * dt * V`

**Question 1.** Assuming `sqrt` and `div` count for one flop each, and assuming `x, y, z` and `u, v, w` are **double-precision** floating point
numbers, **estimate the arithmetic intensity of a *particle time step***.  You should ignore the time it takes to load `dt`.  Your answer should have units of flops / byte.  Give your answer in a new cell below this one, and show how you arrived at that number.

**Answer:**  
    1. 3 ADD, 4 MUL, 6 loads, 3 writes: AI= 7/(9*8) = 7/72 = 0.097 
    2. 3 MUL, 3 loads, 1 writes: AI = 3/4/8 = 3/32 = 0.094
    3. 1 sqrt, 1 load, 1 write: AI = 1 / 2 / 8 = 1/16 = 0.0625
    4. 1 MUL, 1 DIV, 2 loads, 1 write: AI = 2 / 3/8 = 1/12 = 0.083
    5. 6 MUL, 3 ADD, 7 loads, 3 writes: AI = 9 / 10/8 = 9/80 = 0.1125
    6. ... (return to 1) 

**Question 2.** Using the peak theoretical **double-precision** flop/s of this node (flop/s on the CPUs and GPU combined), calculated the same way as in the last assignment, and reported peak memory bandwidths from the manufacturers, **estimate the system balance of CPUs and the GPU of this node separately**.  Note that the bandwidth estimate from intel will be for one socket (4 cores) with attached memory, and our node has two such sockets.

**Answer:**  
Peak theoretical double-precision flop/s of this node is 1638(CPU) and 4763 GFLOPS. 
This server with DDR4 2133 RAM has 68.3 GB/s * 2 sockets = 136.6GB/s bandwidth. Peak GPU meory bandwidth is 720 GB/s.  
Therefore, the system balance for CPU is:  
$1638 GFLOPS / 136.6GB/s = 11.99 flop / byte $  
the system balance for GPU is:  
$4763 GFLOPS / 720GB/s = 6.61 flop / byte $



Last week, we didn't take the peak flop/s values from the manufacturers at face value, and this week we are not going to take the beak Gbyte/s for granted either.  Last week we used a custom benchmark in our calculations; this week we will use an industry standard: the
[STREAM benchmark](https://www.cs.virginia.edu/stream/ref.html).

We can run the stream benchmark on the CPUs for this assignment with a makefile target:

In [16]:
!make runstream STREAM_N=40000000

icc -g -Wall -fPIC -O -qopt-report=3 -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -o stream stream.c -DSTREAM_ARRAY_SIZE=40000000
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
./stream
-------------------------------------------------------------
STREAM version $Revision: 5.10 $
-------------------------------------------------------------
This system uses 8 bytes per array element.
-------------------------------------------------------------
Array size = 40000000 (elements), Offset = 0 (elements)
Memory per array = 305.2 MiB (= 0.3 GiB).
Total memory required = 915.5 MiB (= 0.9 GiB).
Each kernel will be executed 10 times.
 The *best* time for each kernel (excluding the first iteration)
 will be used to compute the reported bandwidth.
-------------------------------------------------------------
Number of Threads requested = 1
Number of Threads counted = 1
-------------------------------------------------------------
Your clock

The `STREAM_N` argument will control the size of the stream arrays.

**Question 3:** Modify the invocation of `make runstreams` by modifying the values of
`STREAM_N`, `COPTFLAGS` (optimization flags), `OMP_NUM_THREADS` and/or `OMP_PROC_BIND` (the openMP environment variables) to get the largest streaming bandwidth from main memory that you can for this node.

[The OpenMP environment variables were not defined by me in the Makefile: they are environment variables that will be detected by the OpenMP runtime when an OpenMP program begins.  You should put them _before_ the make command, e.g. `OMP_NUM_THREADS=5 make runstream STREAM_N=40000000`]

- Follow the directions in the output of the file and make sure you are testing streaming bandwidth from memory and not from a higher level of cache.
- You should try to get close to the same bandwidth for all tests:

- There are two variables in the openMP environment you should care about, OMP_NUM_THREADS, which is self explanatory, and OMP_PROC_BIND is discussed [here](http://pages.tacc.utexas.edu/~eijkhout/pcse/html/omp-affinity.html).  **You should try to use as few threads as possible** to achieve peak bandwidth.

In [138]:
!OMP_NUM_THREADS=32 OMP_PROC_BIND=True make runstream STREAM_N=4000000 -o3

icc -g -Wall -fPIC -O -qopt-report=3 -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -o stream stream.c -DSTREAM_ARRAY_SIZE=4000000
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
./stream
-------------------------------------------------------------
STREAM version $Revision: 5.10 $
-------------------------------------------------------------
This system uses 8 bytes per array element.
-------------------------------------------------------------
Array size = 4000000 (elements), Offset = 0 (elements)
Memory per array = 30.5 MiB (= 0.0 GiB).
Total memory required = 91.6 MiB (= 0.1 GiB).
Each kernel will be executed 10 times.
 The *best* time for each kernel (excluding the first iteration)
 will be used to compute the reported bandwidth.
-------------------------------------------------------------
Number of Threads requested = 32
Number of Threads counted = 32
-------------------------------------------------------------
Your clock g

**Question 4:** What does `OMP_PROC_BIND=close` mean, and why is it a bad choice, not just for this benchmark, but for any streaming kernel?

**Answer:**  
The value OMP_PROC_BIND=close means that the assignment goes successively through the available places. It's bad because when the worker thread is not kept in the same place partition, accessing and updating data becomes more expensive due to the distance. 

**Question 5:** I've modified the benchmark, calling it `stream2.c`.  Here's the difference, it's one line of code:

In [12]:
!diff stream.c stream2.c

267d266
< #pragma omp parallel for


Copy your options for `runstream` to `runstream2` below.  The reported results should be different: why?

In [129]:
!OMP_NUM_THREADS=32 OMP_PROC_BIND=True  make runstream2 STREAM_N=4000000 -o3

icc -g -Wall -fPIC -O -qopt-report=3 -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -o stream2 stream2.c -DSTREAM_ARRAY_SIZE=4000000
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
./stream2
-------------------------------------------------------------
STREAM version $Revision: 5.10 $
-------------------------------------------------------------
This system uses 8 bytes per array element.
-------------------------------------------------------------
Array size = 4000000 (elements), Offset = 0 (elements)
Memory per array = 30.5 MiB (= 0.0 GiB).
Total memory required = 91.6 MiB (= 0.1 GiB).
Each kernel will be executed 10 times.
 The *best* time for each kernel (excluding the first iteration)
 will be used to compute the reported bandwidth.
-------------------------------------------------------------
Number of Threads requested = 32
Number of Threads counted = 32
-------------------------------------------------------------
Your cloc

**Answer:**  
When line 267 `#pragma omp parallel for` is commented, loop `Get initial value for system clock` is not using multi prosessing and thus the performance drops. 

**Question 6:** Now we're going to run stream benchmarks for the GPU.  As above, modify the array size until you believe you are testing streaming bandwidth from memory and not from cache.

In [25]:
!make runstreamcu STREAM_N=4000000

nvcc -ccbin=icpc -lineinfo -Xcompiler '-fPIC' -O -o streamcu stream.cu -DSTREAM_ARRAY_SIZE=4000000
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
./streamcu
-------------------------------------------------------------
CSE6230 CUDA STREAM based on version $Revision: 5.10 $
-------------------------------------------------------------
This system uses 8 bytes per array element.
-------------------------------------------------------------
Array size = 4000000 (elements), Offset = 0 (elements)
Memory per array = 30.5 MiB (= 0.0 GiB).
Total memory required = 91.6 MiB (= 0.1 GiB).
Each kernel will be executed 10 times.
 The *best* time for each kernel (excluding the first iteration)
 will be used to compute the reported bandwidth.
Device Number: 0
  Device name: Tesla P100-PCIE-16GB
  Memory Clock Rate (KHz): 715000
  Memory Bus Width (bits): 4096
  Peak Memory

**Question 7 (2 pts):** This is final time we're running a stream benchmark, I promise.  This benchmark is also for the GPU, but instead of the arrays originating in the GPUs memory, they start on the CPUs memory, and must be transfered to the GPU and back.  This mimics a common design pattern when people try to modify their code for GPUs: identify the bottleneck kernel, and try to "offload" it to the GPU, where it will have a higher throughput (once it get's there).  You don't have to modify this run, I just want you to see what bandwidths it reports:

In [27]:
!make runstreamcu2 STREAM_N=1000000

nvcc -ccbin=icpc -lineinfo -Xcompiler '-fPIC' -O -o streamcu2 stream2.cu -DSTREAM_ARRAY_SIZE=1000000
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
stream2.cu(569): warning: variable "i" was set but never used

stream2.cu(569): warning: variable "i" was set but never used

./streamcu2
-------------------------------------------------------------
CSE6230 CUDA STREAM based on version $Revision: 5.10 $
-------------------------------------------------------------
This system uses 8 bytes per array element.
-------------------------------------------------------------
Array size = 1000000 (elements), Offset = 0 (elements)
Memory per array = 7.6 MiB (= 0.0 GiB).
Total memory required = 22.9 MiB (= 0.0 GiB).
Each kernel will be executed 10 times.
 The *best* time for each kernel (excluding the first iteration)
 will be used to compute the reported bandwidth.
Ordi

Now, with the three peak bandwidths that we have *computed* (not the reported values from question 2) -- CPU, GPU with arrays on the GPU, and GPU with arrays on the CPU -- and with the theoretical peak flop/s for the CPU and GPU, compute *effective system balances* and create a plot with rooflines for all three balances overlayed.

- The y axis should be absolute Gflop/s, not relative, so we can compare them, and should be labeled "Gflop/s"
- Label with roofline goes with which balance: "CPU", "GPU", "CPU->GPU->CPU"
- The x axis should be in units of "double precision flops / byte"

Save your plot as the jpg `threerooflines.jpg` so that it can embed in the cell below

![Three rooflines](./threerooflines.png)

**Question 8 (2 pts):** Remember those particles all the way back in question 1?  Your arithmetic intensity estimate could be placed on the roofline plot for the CPUs, and you could make a judgement about whether the kernel is compute bound or memory bound.

Now let's put it to the test.  The `make runcloud` target simulates `NPOINT` particles orbiting the sun for `NT` time steps.  Because these particles are independent, you can optionally "chunk" multiple time steps for each particle independent of the other particles.  Doing this reduces the number of memory accesses per flop:  each particle stays in register for `NCHUNK` time steps.

Do your best to optimize the throughput of the simulation both in the limit of few particles and many time steps, and in the limit of many particles and few time steps.
Do that by modifying the commands below.

- Make the simulations each run about a second
- Do your best to optimize the compiler flags and the runtime (openMP) environment

Using the outputs of those runs, estimate the floating point efficiency of our particle-time-step kernel: compare the peak flop/s of the CPU, to the product of particle time steps per second and your estimate of the flops per particle time step. and divide by the throughput of particle time steps per second.  Give that effective arithmetic intensity below.

In [118]:
!OMP_NUM_THREADS=32 OMP_PROC_BIND=True OMP_PLACES=cores make runcloud NPOINT=64 NT=4000000 NCHUNK=1000 -o3

rm -f cloud cloud.o verlet.o
make verlet.o DEFINES="-DNT=1000"
make[1]: Entering directory `/nv/coc-ice/zjiang333/cse6230-hw/3-streams-and-roofs'
icc -std=c99 -g -Wall -fPIC -O -qopt-report=3 -I/usr/local/pacerepov1/cuda/8.0.44/include -DNT=1000 -qopenmp -c -o verlet.o verlet.c
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
make[1]: Leaving directory `/nv/coc-ice/zjiang333/cse6230-hw/3-streams-and-roofs'
make cloud
make[1]: Entering directory `/nv/coc-ice/zjiang333/cse6230-hw/3-streams-and-roofs'
icc -std=c99 -g -Wall -fPIC -O -qopt-report=3 -I/usr/local/pacerepov1/cuda/8.0.44/include  -qopenmp -c -o cloud.o cloud.c
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
icpc -qopenmp -o cloud verlet.o cloud.o -Wl,-rpath,.
make[1]: Leaving directory `/nv/coc-ice/zjiang333/cse6230-hw/3-streams-and-roofs'
./cloud 64 4000000 0.01 1000
./cloud, NUM_POINTS=64, NUM_STEPS=4000000, DT=0.01, NCHUNK=1000
[./

In [123]:
!OMP_NUM_THREADS=32 OMP_PROC_BIND=True OMP_PLACES=cores make runcloud NPOINT=6400000 NT=100 NCHUNK=2 -o3

rm -f cloud cloud.o verlet.o
make verlet.o DEFINES="-DNT=2"
make[1]: Entering directory `/nv/coc-ice/zjiang333/cse6230-hw/3-streams-and-roofs'
icc -std=c99 -g -Wall -fPIC -O -qopt-report=3 -I/usr/local/pacerepov1/cuda/8.0.44/include -DNT=2 -qopenmp -c -o verlet.o verlet.c
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
make[1]: Leaving directory `/nv/coc-ice/zjiang333/cse6230-hw/3-streams-and-roofs'
make cloud
make[1]: Entering directory `/nv/coc-ice/zjiang333/cse6230-hw/3-streams-and-roofs'
icc -std=c99 -g -Wall -fPIC -O -qopt-report=3 -I/usr/local/pacerepov1/cuda/8.0.44/include  -qopenmp -c -o cloud.o cloud.c
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
icpc -qopenmp -o cloud verlet.o cloud.o -Wl,-rpath,.
make[1]: Leaving directory `/nv/coc-ice/zjiang333/cse6230-hw/3-streams-and-roofs'
./cloud 6400000 100 0.01 2
./cloud, NUM_POINTS=6400000, NUM_STEPS=100, DT=0.01, NCHUNK=2
[./cloud]: 7.

**Answer:**  
Theoritical CPU peak flops is:   
$8 (processor) * 4 (cores) * 3.20 GHz (frequeny, w/ turbo boost) * 32 (FLOPS/clock) = 3,276.8 GFLOPS$   
and effective FLOPS with `NPOINT=6400000 NT=100` is:   
$8.4 e^8 \times 21 = 17.64 GFLOPS$.   
The effective arithmetic intensity is:  
$21/(72+32+16+24+80) * 2 = 0.1875$